In [1]:
import sys

In [2]:
sys.path[0] = "d:\\projects\\affect\\TUCORE-GCN"
sys.path

['d:\\projects\\affect\\TUCORE-GCN',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311\\DLLs',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311\\Lib',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv',
 '',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\win32',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\win32\\lib',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\Pythonwin']

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer
from models.BERT.TUCOREGCN_BERT import TUCOREGCN_BERT, BertConfig
from data import TUCOREGCNDataset, TUCOREGCNDataloader
from transformers.pipelines import pipeline
import pickle

d:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


d:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\dgl\dgl.dll


In [4]:
device = torch.device("cuda:0")

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
tucore_gcn_bert = TUCOREGCN_BERT(BertConfig.from_json_file("../pre-trained_model/BERT/bert_config.json"), num_labels=36)

In [7]:
tucore_gcn_bert.load_state_dict(torch.load("../TUCOREGCN_BERT_DialogRE/model_best.pt", map_location="cuda"))
tucore_gcn_bert.cuda()

TUCOREGCN_BERT(
  (bert): BertModel(
    (embeddings): BERTEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (speaker_embeddings): Embedding(512, 768)
      (LayerNorm): BERTLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BERTEncoder(
      (layer): ModuleList(
        (0-11): 12 x BERTLayer(
          (attention): BERTAttention(
            (self): BERTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BERTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BERTLayerNorm()
              (dropout): Dropout

In [8]:
test_dialogue = torch.LongTensor([tokenizer.convert_tokens_to_ids(tokenizer.tokenize("[CLS]Hi! I'm Flowey, Flowey the flower![SEP]"))])
test_speaker = torch.LongTensor([0]*len(test_dialogue))

In [9]:
dev_set = TUCOREGCNDataset(src_file="../datasets/DialogRE/dev.json", save_file="../datasets/DialogRE/dev_BERT.pkl", max_seq_length=512, tokenizer=tokenizer, n_class=36, encoder_type='BERT')
dev_loader = TUCOREGCNDataloader(dataset=dev_set, batch_size=1, shuffle=False, relation_num=36, max_length=512)


Reading data from ../datasets/DialogRE/dev.json.
load preprocessed data from ../datasets/DialogRE/dev_BERT.pkl.


In [10]:
idx, batch = next(enumerate(dev_loader))
batch

{'input_ids': tensor([[  101,     3,  1024,  4931,   999,  5882,  1016,  1024,  4931,  1012,
           5882,  1017,  1024,  4931,  1010,  2158,  1012,  2054,  1005,  1055,
           2039,  1029,     3,  1024,  2672,  2017,  2064,  2425,  2033,  1012,
           2026,  4005,  2052,  2066,  2000,  2113,  2339,  1045,  2134,  1005,
           1056,  2265,  2039,  2012,  1996, 14597,  1045,  2134,  1005,  1056,
           2113,  1045,  2018,  2651,  1012,  1996,  2034,  2204,  2518,  2016,
           4152,  2033,  1999,  3134,  1012,  2129,  2071,  2017,  2025,  2507,
           2033,  1996,  4471,  1029,   999,  5882,  1017,  1024,  2092,  1010,
           1045,  1005,  2222,  2425,  8038,  1045,  2079,  5959,  8056,  1010,
           2021,  1010,  6289,  1010,  2009,  2347,  1005,  1056,  2033,  1012,
           5882,  1016,  1024,  2748,  1010,  2009,  2001,   999,  2009,  2001,
           2032,   999,  7910,  9616,   999,  3100,  1010,  2009,  2001,  2033,
            999,     3,  10

In [23]:
entry_idx = 0
input_ids = batch["input_ids"].to("cuda:0")
segment_ids = batch["segment_ids"].to("cuda:0")
input_mask = batch["input_masks"].to("cuda:0")
speaker_ids = batch["speaker_ids"].to("cuda:0")
graphs = batch["graphs"]
mention_id = batch["mention_ids"].to("cuda:0")
turn_mask = batch["turn_masks"].to("cuda:0")
logits = tucore_gcn_bert(
	input_ids,
	segment_ids,
	input_mask,
	speaker_ids,
	graphs,
	mention_id,
	None,
	turn_mask
)

cuda:0


In [25]:
logits

tensor([[ -4.3505,  -2.7431,  -3.9807,  -3.2567,  -6.0214,  -3.5813,  -1.4219,
          -4.4033,  -4.2782,  -4.9335,  -3.5914,  -6.0117,  -2.1588,  -3.8887,
          -2.5400,  -4.5949,  -2.5093,  -6.7579, -12.2926,  -7.4864,  -9.5331,
          -7.5495,  -8.5769,  -6.3480,  -8.5716,  -7.8853, -10.4961,  -5.8990,
          -8.7028,  -6.0993,  -7.7270,  -7.2681, -12.7441,  -5.5965,  -6.7239,
          -5.9486]], device='cuda:0', grad_fn=<ViewBackward0>)

In [24]:
logits.argmax(1)

tensor([6], device='cuda:0')

In [65]:
sequence_outputs, pooled_outputs = tucore_gcn_bert.bert(test_dialogue, test_speaker)

In [66]:
sequence_outputs

tensor([[[-0.2006,  0.1525,  0.4920,  ..., -0.0985, -0.3281, -0.0443],
         [ 0.3134,  0.0418,  0.7631,  ...,  0.9421, -0.6366,  0.4469],
         [ 0.1686,  0.0200,  0.4321,  ...,  0.3830, -0.4554, -0.3702],
         ...,
         [-1.1565,  0.3951,  0.5653,  ...,  0.3693, -0.8601,  0.6407],
         [-0.4329,  0.6883,  0.4808,  ...,  0.0828, -0.3398, -0.2907],
         [ 0.8026, -0.3991, -0.7652,  ...,  0.6569, -0.4829, -0.7216]]],
       grad_fn=<AddBackward0>)

In [67]:
SDPA_DROPOUT = nn.Dropout(0.1)

In [92]:
def sdpa_forward(temperature, q, k, v):
    attn = torch.matmul(q / temperature, k.transpose(2, 3))
    attn = SDPA_DROPOUT(F.softmax(attn, dim=-1))
    output = torch.matmul(attn, v)
    return output, attn

In [108]:
Q = sequence_outputs
K,V = Q,Q
Q,K,V

(tensor([[[-0.2006,  0.1525,  0.4920,  ..., -0.0985, -0.3281, -0.0443],
          [ 0.3134,  0.0418,  0.7631,  ...,  0.9421, -0.6366,  0.4469],
          [ 0.1686,  0.0200,  0.4321,  ...,  0.3830, -0.4554, -0.3702],
          ...,
          [-1.1565,  0.3951,  0.5653,  ...,  0.3693, -0.8601,  0.6407],
          [-0.4329,  0.6883,  0.4808,  ...,  0.0828, -0.3398, -0.2907],
          [ 0.8026, -0.3991, -0.7652,  ...,  0.6569, -0.4829, -0.7216]]],
        grad_fn=<AddBackward0>),
 tensor([[[-0.2006,  0.1525,  0.4920,  ..., -0.0985, -0.3281, -0.0443],
          [ 0.3134,  0.0418,  0.7631,  ...,  0.9421, -0.6366,  0.4469],
          [ 0.1686,  0.0200,  0.4321,  ...,  0.3830, -0.4554, -0.3702],
          ...,
          [-1.1565,  0.3951,  0.5653,  ...,  0.3693, -0.8601,  0.6407],
          [-0.4329,  0.6883,  0.4808,  ...,  0.0828, -0.3398, -0.2907],
          [ 0.8026, -0.3991, -0.7652,  ...,  0.6569, -0.4829, -0.7216]]],
        grad_fn=<AddBackward0>),
 tensor([[[-0.2006,  0.1525,  0.4920

In [109]:
N_HEAD = 12
M_SIZE = 768
K_SIZE = int(768/12)
V_SIZE = int(768/12)
TEMPERATURE = K_SIZE ** 0.5

sz_b, len_q, len_k, len_v = Q.size(0), Q.size(1), K.size(1), V.size(1)
residual = Q

In [110]:
Q.view(sz_b, len_q, N_HEAD, K_SIZE).shape

torch.Size([1, 15, 12, 64])

In [111]:
Q.view(sz_b, len_q, N_HEAD, K_SIZE).transpose(1, 2).shape

torch.Size([1, 12, 15, 64])

In [112]:
q = Q.view(sz_b, len_q, N_HEAD, K_SIZE).transpose(1, 2)
k = K.view(sz_b, len_k, N_HEAD, K_SIZE).transpose(1, 2)
v = V.view(sz_b, len_v, N_HEAD, V_SIZE).transpose(1, 2)

In [113]:
q

tensor([[[[-0.2006,  0.1525,  0.4920,  ..., -0.2199,  0.0624, -0.3040],
          [ 0.3134,  0.0418,  0.7631,  ..., -0.6378, -0.4597, -0.1262],
          [ 0.1686,  0.0200,  0.4321,  ..., -0.6865,  0.1268,  0.4269],
          ...,
          [-1.1565,  0.3951,  0.5653,  ..., -0.7774, -0.1663, -0.9071],
          [-0.4329,  0.6883,  0.4808,  ..., -0.4828,  0.5765, -0.4630],
          [ 0.8026, -0.3991, -0.7652,  ..., -0.5097,  0.6836, -0.4744]],

         [[ 0.5684,  0.0188, -0.0326,  ...,  0.2229,  0.0637,  0.1624],
          [ 1.1151,  0.1873,  0.3643,  ...,  0.1153,  0.5826,  1.2940],
          [-0.7175,  0.6431,  0.1659,  ...,  0.5098,  0.2153,  0.1611],
          ...,
          [ 1.1745,  0.4817,  0.7727,  ...,  1.9266, -0.2642,  1.2091],
          [-0.2577,  0.3078,  0.5520,  ...,  0.4616,  0.1806,  0.4137],
          [ 0.4694,  1.3367, -0.8174,  ...,  0.8301, -0.9760, -0.3622]],

         [[ 0.1766, -0.1704,  0.0158,  ...,  0.5301, -0.3743, -0.0368],
          [ 0.3716,  0.1228,  

In [106]:
q, attn = sdpa_forward(TEMPERATURE, q, k, v)

In [107]:
q

tensor([[[[-0.0233, -0.0534,  0.2859,  ..., -0.4817,  0.0962, -0.0733],
          [ 0.1556, -0.0062,  0.5206,  ..., -0.5524, -0.1295, -0.0493],
          [ 0.0332, -0.0585,  0.3798,  ..., -0.5612,  0.0697,  0.0855],
          ...,
          [-0.3925,  0.0644,  0.3694,  ..., -0.6206,  0.0074, -0.3413],
          [-0.0812,  0.0196,  0.3462,  ..., -0.5348,  0.1730, -0.0779],
          [ 0.7348, -0.3897, -0.6663,  ..., -0.5537,  0.6545, -0.4399]],

         [[ 0.4249,  0.3681,  0.0167,  ...,  0.4800,  0.1476,  0.2384],
          [ 0.8529,  0.3746,  0.2464,  ...,  0.4189,  0.3877,  0.7853],
          [ 0.1192,  0.5297,  0.1609,  ...,  0.5179,  0.2174,  0.2897],
          ...,
          [ 0.7541,  0.4675,  0.3605,  ...,  1.0866, -0.0232,  0.6736],
          [ 0.3347,  0.4776,  0.2096,  ...,  0.5629,  0.1878,  0.3301],
          [ 0.5412,  1.1516, -0.5498,  ...,  0.8206, -0.6543, -0.1567]],

         [[ 0.7262, -0.3649,  0.0430,  ...,  0.4110, -0.4739,  0.4733],
          [ 0.5596, -0.1258,  

In [186]:
q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1)
q = F.dropout(q)
q += residual
q

tensor([[[ -0.2006,   0.7793,   0.4920,  ...,  -5.0510,  -0.3281,   1.3801],
         [  8.9824,   0.2507,   0.7631,  ..., -21.3476,  -6.6887,  -6.6413],
         [ -0.0618,   1.6801,   3.0003,  ...,   0.3830,  -0.6904,  -0.3702],
         ...,
         [ -0.1799,   0.3951,   0.5653,  ...,  -2.7048,  -0.8601,   0.6407],
         [ -1.4788,   1.7556,  -5.6796,  ...,   0.0828,  -1.2728,  -0.2907],
         [ 10.8724,   0.4165,  -2.6196,  ...,   0.6569,   2.3774,  -0.7216]]],
       grad_fn=<AddBackward0>)

In [ ]:
k_size, v_size, n_head = self.k_size, self.v_size, self.n_head
sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)

residual = q

# Pass through the pre-attention projection: b x lq x (n*dv)
# Separate different heads: b x lq x n x dv
q = self.w_qs(q).view(sz_b, len_q, n_head, k_size)
k = self.w_ks(k).view(sz_b, len_k, n_head, k_size)
v = self.w_vs(v).view(sz_b, len_v, n_head, v_size)

# Transpose for attention dot product: b x n x lq x dv
q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

if mask is not None:
	mask = mask.unsqueeze(1)   # For head axis broadcasting.

q, attn = self.attention(q, k, v, mask=mask)

# Transpose to move the head dimension back: b x lq x n x dv
# Combine the last two dimensions to concatenate all the heads together: b x lq x (n*dv)
q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1)
q = self.dropout(self.fc(q))
q += residual

q = self.layer_norm(q)


In [ ]:


w_qs = F.linear(M_SIZE, N_HEAD * K_SIZE, bias=False)
w_ks = nn.Linear(M_SIZE, N_HEAD * K_SIZE, bias=False)
w_vs = nn.Linear(M_SIZE, N_HEAD * V_SIZE, bias=False)
fc = nn.Linear(N_HEAD * V_SIZE, M_SIZE, bias=False)


#attention


In [ ]:
class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, k_size, v_size, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.k_size = k_size
        self.v_size = v_size

        self.w_qs = nn.Linear(d_model, n_head * k_size, bias=False)
        self.w_ks = nn.Linear(d_model, n_head * k_size, bias=False)
        self.w_vs = nn.Linear(d_model, n_head * v_size, bias=False)
        self.fc = nn.Linear(n_head * v_size, d_model, bias=False)

        self.attention = ScaledDotProductAttention(temperature=k_size ** 0.5)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, q, k, v, mask=None):

        k_size, v_size, n_head = self.k_size, self.v_size, self.n_head
        sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)

        residual = q

        # Pass through the pre-attention projection: b x lq x (n*dv)
        # Separate different heads: b x lq x n x dv
        q = self.w_qs(q).view(sz_b, len_q, n_head, k_size)
        k = self.w_ks(k).view(sz_b, len_k, n_head, k_size)
        v = self.w_vs(v).view(sz_b, len_v, n_head, v_size)

        # Transpose for attention dot product: b x n x lq x dv
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1)   # For head axis broadcasting.

        q, attn = self.attention(q, k, v, mask=mask)

        # Transpose to move the head dimension back: b x lq x n x dv
        # Combine the last two dimensions to concatenate all the heads together: b x lq x (n*dv)
        q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1)
        q = self.dropout(self.fc(q))
        q += residual

        q = self.layer_norm(q)

        return q, attn